In [ ]:
pip install ipython-sql pymysql

In [4]:
# Load the SQL extension for Jupyter Notebook
%load_ext sql

# Connect to the SQLite database
%sql sqlite:///ReactIR.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [5]:
%%sql

-- Create Users table
CREATE TABLE IF NOT EXISTS Users (
    UserID INTEGER PRIMARY KEY,
    Username TEXT NOT NULL UNIQUE
);

-- Create Projects table
CREATE TABLE IF NOT EXISTS Projects (
    ProjectID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    UserID INTEGER,
    FOREIGN KEY (UserID) REFERENCES Users(UserID)
);

-- Create Experiments table
CREATE TABLE IF NOT EXISTS Experiments (
    ExperimentID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    ProjectID INTEGER,
    FOREIGN KEY (ProjectID) REFERENCES Projects(ProjectID)
);

-- Create Documents table
CREATE TABLE IF NOT EXISTS Documents (
    DocumentID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    ExperimentID INTEGER,
    FOREIGN KEY (ExperimentID) REFERENCES Experiments(ExperimentID)
);

-- Create Probes table
CREATE TABLE IF NOT EXISTS Probes (
    ProbeID INTEGER PRIMARY KEY,
    Description TEXT NOT NULL,
    DocumentID INTEGER,
    LatestTemperatureCelsius REAL,
    LatestTemperatureTime TEXT,
    FOREIGN KEY (DocumentID) REFERENCES Documents(DocumentID)
);

-- Create Samples table
CREATE TABLE IF NOT EXISTS Samples (
    SampleID INTEGER PRIMARY KEY,
    ProbeID INTEGER,
    SampleCount INTEGER,
    LastSampleTime TEXT,
    CurrentSamplingInterval INTEGER,
    MinSamplingInterval INTEGER,
    MaxSamplingInterval INTEGER,
    FOREIGN KEY (ProbeID) REFERENCES Probes(ProbeID)
);

-- Create Spectra table
CREATE TABLE IF NOT EXISTS Spectra (
    SpectraID INTEGER PRIMARY KEY,
    SampleID INTEGER,
    Type TEXT CHECK (Type IN ('raw', 'background')),
    FilePath TEXT NOT NULL,
    RecordedAt TEXT,
    FOREIGN KEY (SampleID) REFERENCES Samples(SampleID)
);


 * sqlite:///ReactIR.db
   sqlite:///chemistry.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]